In [21]:
import math
from typing import List, NamedTuple, Optional
import numpy
import cv2
from dataclasses import dataclass

def mean_pixel_distance(left: numpy.ndarray, right: numpy.ndarray) -> float:
    """Return the mean average distance in pixel values between `left` and `right`.
    Both `left and `right` should be 2 dimensional 8-bit images of the same shape.
    """
    assert len(left.shape) == 2 and len(right.shape) == 2
    assert left.shape == right.shape
    num_pixels: float = float(left.shape[0] * left.shape[1])
    return (numpy.sum(numpy.abs(left.astype(numpy.int32) - right.astype(numpy.int32))) / num_pixels)


def estimated_kernel_size(frame_width: int, frame_height: int) -> int:
    """Estimate kernel size based on video resolution."""
    # TODO: This equation is based on manual estimation from a few videos.
    # Create a more comprehensive test suite to optimize against.
    size: int = 4 + round(math.sqrt(frame_width * frame_height) / 192)
    if size % 2 == 0:
        size += 1
    return size
class ContentDetector():
    class Components(NamedTuple):
        """Components that make up a frame's score, and their default values."""
        delta_hue: float = 1.0
        """Difference between pixel hue values of adjacent frames."""
        delta_sat: float = 1.0
        """Difference between pixel saturation values of adjacent frames."""
        delta_lum: float = 1.0
        """Difference between pixel luma (brightness) values of adjacent frames."""
        delta_edges: float = 1.0
        """Difference between calculated edges of adjacent frames.
        Edge differences are typically larger than the other components, so the detection
        threshold may need to be adjusted accordingly."""

    DEFAULT_COMPONENT_WEIGHTS = Components()
    FRAME_SCORE_KEY = 'content_val'
    """Key in statsfile representing the final frame score after weighed by specified components."""
    @dataclass
    class _FrameData:
        """Data calculated for a given frame."""
        hue: numpy.ndarray
        """Frame hue map [2D 8-bit]."""
        sat: numpy.ndarray
        """Frame saturation map [2D 8-bit]."""
        lum: numpy.ndarray
        """Frame luma/brightness map [2D 8-bit]."""
        edges: Optional[numpy.ndarray]
        """Frame edge map [2D 8-bit, edges are 255, non edges 0]. Affected by `kernel_size`."""
    def __init__(
        self,
        threshold: float = 27.0,
        min_scene_len: int = 15,
        weights: 'ContentDetector.Components' = DEFAULT_COMPONENT_WEIGHTS,
        luma_only: bool = False,
        kernel_size: Optional[int] = None,
    ):
        self._threshold: float = threshold
        self._min_scene_len: int = min_scene_len
        self._last_scene_cut: Optional[int] = None
        self._last_frame: Optional[ContentDetector._FrameData] = None
        self._weights: ContentDetector.Components = weights
        if luma_only:
            self._weights = ContentDetector.LUMA_ONLY_WEIGHTS
        self._kernel: Optional[numpy.ndarray] = None
        if kernel_size is not None:
            print(kernel_size)
            if kernel_size < 3 or kernel_size % 2 == 0:
                raise ValueError('kernel_size must be odd integer >= 3')
            self._kernel = numpy.ones((kernel_size, kernel_size), numpy.uint8)
        self._frame_score: Optional[float] = None
    def _calculate_frame_score(self, frame_num: int, frame_img: numpy.ndarray) -> float:
        """Calculate score representing relative amount of motion in `frame_img` compared to
        the last time the function was called (returns 0.0 on the first call)."""
        # TODO: Add option to enable motion estimation before calculating score components.
        # TODO: Investigate methods of performing cheaper alternatives, e.g. shifting or resizing
        # the frame to simulate camera movement, using optical flow, etc...

        # Convert image into HSV colorspace.
        hue, sat, lum = cv2.split(cv2.cvtColor(frame_img, cv2.COLOR_BGR2HSV))

        # Performance: Only calculate edges if we have to.
        calculate_edges: bool = (self._weights.delta_edges > 0.0)
#                                  or (self.stats_manager is not None)
        edges = self._detect_edges(lum) if calculate_edges else None

        if self._last_frame is None:
            # Need another frame to compare with for score calculation.
            self._last_frame = ContentDetector._FrameData(hue, sat, lum, edges)
            return 0.0

        score_components = ContentDetector.Components(
            delta_hue=mean_pixel_distance(hue, self._last_frame.hue),
            delta_sat=mean_pixel_distance(sat, self._last_frame.sat),
            delta_lum=mean_pixel_distance(lum, self._last_frame.lum),
            delta_edges=(0.0 if edges is None else mean_pixel_distance(
                edges, self._last_frame.edges)),
        )
#         print(score_components.delta_edges)
#         print(score_components.delta_hue)
#         print(score_components.delta_sat)
#         print(score_components.delta_lum)
        

        frame_score: float = (
            sum(component * weight for (component, weight) in zip(score_components, self._weights))
            / sum(abs(weight) for weight in self._weights))

#         # Record components and frame score if needed for analysis.
#         if self.stats_manager is not None:
#             metrics = {self.FRAME_SCORE_KEY: frame_score}
#             metrics.update(score_components._asdict())
#             self.stats_manager.set_metrics(frame_num, metrics)

        # Store all data required to calculate the next frame's score.
        self._last_frame = ContentDetector._FrameData(hue, sat, lum, edges)
        return frame_score
    def process_frame(self, frame_num: int, frame_img: numpy.ndarray) -> List[int]:
        """ Similar to ThresholdDetector, but using the HSV colour space DIFFERENCE instead
        of single-frame RGB/grayscale intensity (thus cannot detect slow fades with this method).
        Arguments:
            frame_num: Frame number of frame that is being passed.
            frame_img: Decoded frame image (numpy.ndarray) to perform scene
                detection on. Can be None *only* if the self.is_processing_required() method
                (inhereted from the base SceneDetector class) returns True.
        Returns:
            List of frames where scene cuts have been detected. There may be 0
            or more frames in the list, and not necessarily the same as frame_num.
        """
        if frame_img is None:
            # TODO(v0.6.2): Make frame_img a required argument in the interface. Log a warning
            # that passing None is deprecated and results will be incorrect if this is the case.
            return []

        # Initialize last scene cut point at the beginning of the frames of interest.
        if self._last_scene_cut is None:
            self._last_scene_cut = frame_num

        self._frame_score = self._calculate_frame_score(frame_num, frame_img)
        if self._frame_score is None:
            return []

        # We consider any frame over the threshold a new scene, but only if
        # the minimum scene length has been reached (otherwise it is ignored).
        if self._frame_score >= self._threshold and (
            (frame_num - self._last_scene_cut) >= self._min_scene_len):
            self._last_scene_cut = frame_num
            return [frame_num]

        return []


    #def post_process(self, frame_num):
    #    """
    #    return []

    def _detect_edges(self, lum: numpy.ndarray) -> numpy.ndarray:
        """Detect edges using the luma channel of a frame.
        Arguments:
            lum: 2D 8-bit image representing the luma channel of a frame.
        Returns:
            2D 8-bit image of the same size as the input, where pixels with values of 255
            represent edges, and all other pixels are 0.
        """
        # Initialize kernel.
        if self._kernel is None:
            kernel_size = estimated_kernel_size(lum.shape[1], lum.shape[0])
            self._kernel = numpy.ones((kernel_size, kernel_size), numpy.uint8)

        # Estimate levels for thresholding.
        # TODO(v0.6.2): Add config file entries for sigma, aperture/kernel size, etc.
        sigma: float = 1.0 / 3.0
        median = numpy.median(lum)
        low = int(max(0, (1.0 - sigma) * median))
        high = int(min(255, (1.0 + sigma) * median))

        # Calculate edges using Canny algorithm, and reduce noise by dilating the edges.
        edges = cv2.Canny(lum, low, high)
        return cv2.dilate(edges, self._kernel)
    

In [22]:
import cv2
import os

# Set the video file path
video_path = "try.mp4"

# Set the output directory for the frames
output_dir = "frames"

# Set the sampling rate (number of frames to skip between extractions)
sampling_rate = 10

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Open the video file
cap = cv2.VideoCapture(video_path)

# declare object from class 
video_try = ContentDetector()

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Loop through the frames and extract the sampled frames
for frame_idx in range(0, total_frames, sampling_rate):
    # Set the frame index to extract
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)

    # Read the frame from the video
    ret, frame = cap.read()

    # Check if the frame was successfully read
    if ret:
        # Construct the output file path for the frame
        output_path = os.path.join(output_dir, "frame_{:06d}.jpg".format(frame_idx))

        # Write the frame to the output file
        cv2.imwrite(output_path, frame)
    print(video_try.process_frame(frame_idx,frame))
    
# Release the video capture object
cap.release()

[]
33.18541666666667
1.2998090277777778
8.558784722222223
3.9727430555555556
[]
35.434375
1.7501909722222222
11.72734375
4.88921875
[]
45.9796875
2.1058333333333334
12.57451388888889
6.944930555555556
[]
36.43046875
2.777621527777778
12.137829861111111
5.696944444444444
[]
37.475260416666664
2.3113020833333335
12.021041666666667
6.950138888888889
[]
36.09401041666667
2.0861631944444445
11.953559027777779
5.950659722222222
[]
36.35078125
2.036302083333333
11.657222222222222
6.266944444444444
[]
46.04609375
3.6606770833333333
16.169288194444444
11.983732638888888
[]
32.096354166666664
3.3726909722222222
12.944114583333333
9.02576388888889
[]
25.22109375
3.8036631944444443
12.42078125
6.284704861111111
[]
26.079947916666665
4.142413194444444
14.43517361111111
8.049149305555556
[]
21.506770833333334
2.923107638888889
13.772048611111112
8.118402777777778
[]
21.10390625
2.577378472222222
12.543975694444445
7.284288194444445
[]
16.557291666666668
2.564670138888889
11.129947916666667
5.2571180